In [45]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [19]:
train = './birdvsbutterfly/train' 
test = './birdvsbutterfly/test' 

In [20]:
#preprocessing and image augmentation
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = 'true',
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

Found 12111 images belonging to 2 classes.
,Found 3985 images belonging to 2 classes.


In [46]:
model = keras.Sequential([
    #CNN
    tf.keras.layers.Input(shape = (150, 150, 3)),
    tf.keras.layers.Conv2D(32, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.BatchNormalization(),

    #Dense
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer = Adam(learning_rate=0.00035),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [47]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)                   │ (None, 150, 150, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 75, 75, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 75, 75, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 75, 75, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 37, 37, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 37, 37, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 37, 37, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 18, 18, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 18, 18, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 41472)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │       5,308,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,402,817 (20.61 MB)

 Trainable params: 5,402,369 (20.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [48]:
history = model.fit(
    train_generator, 
    steps_per_epoch = 606, 
    validation_data = test_generator,
    validation_steps = 200, 
    epochs = 25
)

Epoch 1/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 267s 437ms/step - accuracy: 0.7665 - loss: 0.7153 - val_accuracy: 0.8635 - val_loss: 0.2959
,Epoch 2/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 270s 445ms/step - accuracy: 0.8412 - loss: 0.3562 - val_accuracy: 0.8735 - val_loss: 0.2945
,Epoch 3/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 270s 446ms/step - accuracy: 0.8595 - loss: 0.3131 - val_accuracy: 0.8349 - val_loss: 0.4143
,Epoch 4/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 269s 444ms/step - accuracy: 0.8753 - loss: 0.2846 - val_accuracy: 0.8344 - val_loss: 0.3890
,Epoch 5/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 270s 445ms/step - accuracy: 0.8832 - loss: 0.2716 - val_accuracy: 0.7471 - val_loss: 0.6406
,Epoch 6/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 271s 447ms/step - accuracy: 0.8944 - loss: 0.2492 - val_accuracy: 0.7358 - val_loss: 0.7071
,Epoch 7/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 271s 448ms/step - accuracy: 0.9041 - loss: 0.2306 - val_accuracy: 0.9132 - val_loss: 0.2351
,Epoch 8/25
,606/606 ━━━━━━━━━━━━━━━━━━━━ 270s 446ms/step - acc

In [62]:
image_path = 'toucan-brightly.webp'

# Load the image and preprocess it
image = load_img(image_path, target_size=(150, 150))  # Resize to the input size
image_array = img_to_array(image)  # Convert to numpy array
image_array = image_array / 255.0  # Normalize to [0, 1]
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(image_array)

# Output prediction
print("Prediction:", prediction)

# For classification, you might decode it:
if prediction[0][0] > 0.5:  # For binary classification
    print("Class: butterfly")
else:
    print("Class: bird")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
,Prediction: [[0.0171577]]
,Class: bird


In [63]:
model.save('final_birdvsbutterfly_model.keras')